<a href="https://colab.research.google.com/github/aiscience-22/UA_War/blob/machine_learning_0.03/Twitter_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done


In [2]:
# Download a Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-30 22:59:41--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.59MB/s    in 0.6s    

2022-08-30 22:59:42 (1.59 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Add drivers to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://uaresources.s3.us-west-1.amazonaws.com/twitter_sentiments.csv"

spark.sparkContext.addFile(url)
text_df = spark.read.csv(SparkFiles.get("twitter_sentiments.csv"), sep=",", header=True, inferSchema=True)


# Show DataFrame
text_df.show()

+---------+---------+-----------+--------------------+-------------------+------------+--------------------+--------+--------------+----------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|following|followers|totaltweets|       usercreatedts|     tweetcreatedts|retweetcount|            hashtags|language|favorite_count|is_retweet|is_quote_status|         extractedts|        cleaned_text|            negative|             neutral|            positive|
+---------+---------+-----------+--------------------+-------------------+------------+--------------------+--------+--------------+----------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       51|     3362|      71331|2011-09-08 02:34:...|2022-08-01 00:00:00|           0|[{'text': 'Russia...|      en|           0.0|     false|          false|2022-08-01 02:30:...|the conflict is b...|  0.

In [7]:
# Drop null values
text_df=text_df.dropna()
text_df.show()

+---------+---------+-----------+--------------------+-------------------+------------+--------------------+--------+--------------+----------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|following|followers|totaltweets|       usercreatedts|     tweetcreatedts|retweetcount|            hashtags|language|favorite_count|is_retweet|is_quote_status|         extractedts|        cleaned_text|            negative|             neutral|            positive|
+---------+---------+-----------+--------------------+-------------------+------------+--------------------+--------+--------------+----------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       51|     3362|      71331|2011-09-08 02:34:...|2022-08-01 00:00:00|           0|[{'text': 'Russia...|      en|           0.0|     false|          false|2022-08-01 02:30:...|the conflict is b...|  0.

In [23]:
# Check data types
text_df.info()

AttributeError: ignored

In [26]:
# Change the dtypes of usercreatedts, tweetcreatedts, and extractedts to datetime64 for easier operation later
text_df["usercreatedts"] = pd.to_datetime(text_df["usercreatedts"])
text_df["tweetcreatedts"] = pd.to_datetime(text_df["tweetcreatedts"])
text_df["extractedts"] = pd.to_datetime(text_df["extractedts"])

# check dtypes
text_df.info()

ValueError: ignored

In [25]:
# Visualize tweet sentiment by date
# get dates in the dataframe 
dates = text_df["tweetcreatedts"].dt.day
# group tweet timestamps by date and get average negative score for each date
negative_by_date = text_df[["tweetcreatedts", "negative"]].groupby(dates).avg('negative')

# plot bar graph of tweet count by date
negative_by_date.plot.line();

plt.title("Negative Tweet Sentiments by Date")
plt.xlabel("Tweet Date")
plt.ylabel("Negative Sentiment")
plt.xticks(rotation=0)
plt.show()

AnalysisException: ignored

**Get our transformed raw data into our database**

In [13]:
# Store environmental variable
from getpass import getpass
password = getpass('password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://tweets.cnzbbvrrhst7.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"uatweets",
          "password": password,
          "driver":"org.postgresql.Driver"}

password··········


In [15]:
# Write DataFrame to active_user table in RDS
text_df.write.jdbc(url=jdbc_url, table='twitter_sentiments.csv', mode=mode, properties=config)

Py4JJavaError: ignored